## Functions, deep dive

### Direct and indirect calls

Имени echo присваивается обьект функции

In [1]:
def echo(message):
    print(message)

Вызов по оригинальному имени

In [2]:
echo('Direct call')

Direct call


Присваиваем обьект функции другой переменной

In [3]:
indirect = echo

Вызов функции не через оригинальное имя

In [4]:
indirect('Indirect call')

Indirect call


### Function attributes

In [5]:
echo

<function __main__.echo(message)>

In [6]:
echo.count = 0

In [7]:
echo.count += 1

In [8]:
echo.count

1

In [9]:
print(dir(echo))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'count']


In [10]:
'count' in dir(echo)

True

### Recursive functions

In [11]:
def mysum(L):
    if not L:
        return 0
    else:
        return L[0] + mysum(L[1:]) # Вызывает саму себя


mysum([1, 2, 3, 4, 5])

15

Fibonacci

In [12]:
def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)
    
print(fib(20))

6765


In [13]:
def fib2(n):
    return 1 if n <= 2 else fib2(n - 1) + fib2(n - 2)

print(fib2(20))

6765


## Task 1

> Написать функцию перевода строки в число, не используя стандартные функции приведения (int, str). Рекурсивно.
Пример: вход - 'abcd', выход - 979899100, тип выхода - int

In [14]:
# TODO

### Lambda functions

In [15]:
def add_one(x):
    return x + 1

print(add_one(2))

3


In [16]:
add_one = lambda x: x + 1

print(add_one(2))

3


In [17]:
(lambda x: x + 1)(2)

3

Closures

In [18]:
x = 10

In [19]:
foo = lambda: x

In [20]:
foo()

10

In [21]:
x = 11

In [22]:
foo()

11

Example

In [23]:
collapse = False
process = collapse and (lambda s: " ".join(s.split())) or (lambda s: s)
process('hello,   world!')

'hello,   world!'

In [24]:
collapse = True
process = collapse and (lambda s: " ".join(s.split())) or (lambda s: s)
process('hello,   world!')

'hello, world!'

### Function closures

In [25]:
def action(x):
    return (lambda y: x + y)

In [26]:
add_5 = action(5)

In [27]:
add_5(2)

7

In [28]:
action(5)(2)

7

### Decorators

#### Trace decorator

In [29]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args,**kwargs)
    return inner

In [30]:
@trace
def useful(x):
    print('I do nothing useful')

In [31]:
useful(42)

useful (42,) {}
I do nothing useful


No help on decorated function

In [32]:
help(useful)

Help on function inner in module __main__:

inner(*args, **kwargs)



Use functools.wraps to pass docstrings and other information about wrapped function

In [33]:
import functools 

def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args,**kwargs)
    return inner

In [34]:
@trace
def useful(x):
    print('I do nothing useful')

In [35]:
help(useful)

Help on function useful in module __main__:

useful(x)



### Decorator with arguments

In [36]:
import functools 
import sys

def trace(output=sys.stdout):
    def decor(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs, file=output)
            return func(*args,**kwargs)
        return inner
    return decor

In [37]:
@trace(sys.stderr)
def useful(x):
    print('I do nothing useful')

In [38]:
useful(42)

I do nothing useful


useful (42,) {}


### with_arguments

In [39]:
def with_arguments(deco):
    @functools.wraps(deco)
    def wrapper(*dargs,**dkwargs):
        def decorator(func):
            result = deco(func,*dargs,**dkwargs)
            functools.update_wrapper(result, func)
            return result
        return decorator
    return wrapper

In [40]:
@with_arguments
def trace(func, output):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, file=output)
        return func(*args,**kwargs)
    return inner

In [41]:
@trace(sys.stderr)
def useful(x):
    print('I do nothing useful')
    
useful(42)

I do nothing useful


useful (42,) {}


## Task 2

> Написать декоратор, который будет считать время работы функции и выводить на экран. Для текущего времени можно использовать модуль time.

### @once decorator

In [42]:
def once(func):
    @functools.wraps(func)
    def inner(*args,**kwargs):
        if not inner.called:
            inner.called = True
            return func(*args,**kwargs)        
    
    inner.called = False
    return inner


In [43]:
@once
def initialize_settings():
    print("Settings initialized.")

In [44]:
initialize_settings()

Settings initialized.


In [45]:
initialize_settings()

### Decorator chainig

In [46]:
def square(func):
    return lambda x: func(x * 2)

def power(func):
    return lambda x: func(x ** 2)

In [47]:
@square
@power
def foo(x):
    print(x)

foo(5)

100


Order matters

In [48]:
@power
@square
def foo(x):
    print(x)

foo(5)

50


## Task 3

> Написать декоратор validate, который будет валидировать входящий аргумент функции на предмет выхода за заданные границы и нужный размер 

In [49]:
def validate(*args, **kwargs):
    raise NotImplementedError('Implement me!')

In [ ]:
@validate(low_bound=0, upper_bound=256)
def set_pixel(pixel_values):
    print("Pixel created!")

In [ ]:
set_pixel((0, 127, 300))

In [ ]:
set_pixel((0, 127, 250))

### Homework

> Необходимо написать фабрику декораторов(также декоратор). Фабрика (функция) принимает аргумент - функцию(lambda) и декоратор. Возвращает декоратор, который должен вызывать функцию(lambda) с аргументом - результатом декорируемого декоратора.